In [1]:
import os
import glob
import pandas as pd
import numpy as np

from transformers4rec import tf as tr
import tensorflow as tf
from transformers4rec.tf.ranking_metric import NDCGAt, RecallAt
from transformers4rec.tf.utils import testing_utils as test_utils
from modules.test_dataloader import get_test_dataset

### Load Test Dataset

In [2]:
# define the input file path
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", '../data/')
# define the output file path
OUTPUT_DIR = os.environ.get("OUTPUT_DIR", "../data/sessions_by_day")
# features chosen to train on
chosen_features = ['product_id-list_seq']
from merlin_standard_lib import Schema
# define schema object to pass it to the TabularSeqeunceFeatures class
SCHEMA_PATH = os.path.join(INPUT_DATA_DIR, 'schema.pb')
schema = Schema().from_proto_text(SCHEMA_PATH)
schema = schema.select_by_name(chosen_features)

In [3]:
test_path = os.path.join(OUTPUT_DIR, f"{1}/train.parquet")
test_df = pd.read_parquet(test_path)
test_df = test_df[['product_id-list_seq']]
test_df = test_df[:100]

test_df,steps = get_test_dataset(test_df,20,len(test_df),['product_id-list_seq'],as_dict=True)

2021-12-21 14:35:58.303683: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Test Dimensionality of TabularSequenceFeature Output

In [4]:
# Input
sequence_length, d_model = 20, 192
# Define input module to process tabular input-features and to prepare masked inputs
inputs = tr.TabularSequenceFeatures.from_schema(
    schema,
    max_sequence_length = sequence_length,
    d_output = d_model,
    masking = 'mlm'
)

In [5]:
outputs = inputs(test_df)

In [6]:
assert outputs.ndim == 3
assert outputs.shape[-1] == d_model
assert outputs.shape[1] == sequence_length